# Cortisol Analysis

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.io import load_long_format_csv
from biopsykit.utils.dataframe_handling import multi_xs

from fau_colors import cmaps

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.io import load_cortisol_samples_log_times
from carwatch_analysis.utils import describe_groups_df
from carwatch_analysis.stats import create_unique_night_id
from carwatch_analysis.plotting import multi_paired_plot_auc, paired_plot_auc


%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)

theme_kwargs = {"context": "talk", "style": "ticks", "palette": palette}
theme_kwargs_grid = {"context": "talk", "style": "ticks", "palette": palette, "font_scale": 0.8}
sns.set_theme(**theme_kwargs)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

export = True

pg.options["round"] = 4

palette

## Setup Paths

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")
result_path = base_path.joinpath("results")
stats_path = result_path.joinpath("statistics")

img_path = result_path.joinpath("plots")

paper_path = Path(json.load(Path("../paper_path.json").open(encoding="utf-8"))["paper_path"])
paper_img_path = paper_path.joinpath("img")

bp.utils.file_handling.mkdirs([result_path, stats_path, img_path, paper_img_path])

## Load Data

### Cortisol Samples

In [ ]:
cort_path = export_path.joinpath("cortisol_samples_processed_all_log_types.csv")
cort_samples = load_cortisol_samples_log_times(cort_path)

cort_samples.head()

### Cortisol Features

In [ ]:
cort_features = load_long_format_csv(export_path.joinpath("cortisol_features_processed_all_log_types.csv"))
cort_features = create_unique_night_id(cort_features)
cort_features.head()

In [ ]:
# Don't consider IMU and IMU_App because the cortisol features are the same as
# Selfreport and App since only the wake onset differs, but not the sampling times
log_types_fine = [
    "Naive",
    "Selfreport without App",
    "Selfreport with App",
    "App",
    "Sensor + Selfreport without App",
    "Sensor + Selfreport with App",
    "Sensor + App",
]
log_types_coarse = ["Naive", "Selfreport", "App", "Sensor + Selfreport", "Sensor + App"]
delay_groups = ["None", "Short", "Moderate", "High"]

log_types_fine_rename = [s.replace("without", "w/o").replace("with", "w/") for s in log_types_fine]
rename_mapper_log_types = dict(zip(log_types_fine, log_types_fine_rename))

log_types_fine = log_types_fine_rename

cort_samples = cort_samples.rename(rename_mapper_log_types, level="log_type")
cort_features = cort_features.rename(rename_mapper_log_types, level="log_type")

## Data Selection

### Cortisol Samples

In [ ]:
cort_samples = multi_xs(cort_samples, log_types_fine, level="log_type")
cort_samples.head()

### Cortisol Features

In [ ]:
cort_features_analysis = cort_features.copy()
cort_features_analysis = multi_xs(cort_features_analysis, ["auc_g", "auc_i"], level="saliva_feature")
cort_features_analysis = multi_xs(cort_features_analysis, log_types_fine, level="log_type")
cort_features_analysis.head()

## Saliva Samples – CAR Plot

In [ ]:
log_types = ["Selfreport w/o App", "App"]

data_plot = cort_samples.reindex(log_types, level="log_type")

car = bp.protocols.CAR()
car.add_saliva_data(saliva_data=data_plot, saliva_type="cortisol", sample_times=[0, 15, 30, 45, 60])

In [ ]:
fig, ax = plt.subplots()

car.car_saliva_plot(
    saliva_type="cortisol",
    hue="log_type",
    style="log_type",
    ax=ax,
    hue_order=log_types,
    x_offset=0.0,
)
fig.tight_layout()

In [ ]:
dict_saliva_export = {}

## Saliva Features – Statistical Analysis

### Log Type

In [ ]:
log_types = ["Naive", "Selfreport w/ App", "App"]

data_analysis = cort_features_analysis.reindex(log_types, level="log_type")

pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("prep", "equal_var"), ("test", "rm_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "cortisol",
        "within": "log_type",
        "subject": "night_id",
        "groupby": "saliva_feature",
        "multicomp": {"method": "bonf", "levels": False},
    },
)

pipeline.apply(data_analysis)
pipeline.export_statistics(stats_path.joinpath("stats_auc_log_type.xlsx"))
pipeline.display_results()

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets(
    "posthoc", stats_effect_type="within", plot_type="single", subplots=True, x="log_type", features=["auc_g", "auc_i"]
)

title_map = {"auc_g": "$AUC_G$", "auc_i": "$AUC_I$"}

fig, axs = plt.subplots(figsize=(12, 5), ncols=2)

for (feature, data), ax in zip(cort_features_analysis.groupby("saliva_feature"), axs):
    bp.protocols.plotting.saliva_feature_boxplot(
        data=data,
        x="log_type",
        saliva_type="cortisol",
        feature=feature,
        order=log_types,
        stats_kwargs={"box_pairs": box_pairs[feature], "pvalues": pvalues[feature], "verbose": 0},
        palette=cmaps.faculties_light,
        ax=ax,
    )
    ax.set_title(title_map[feature], pad=12)
    ax.set_xlabel("Log Type")
    ax.set_ylim(ax.get_ylim()[0], ax.get_ylim()[1] * 1.05)

fig.tight_layout()

for path in [img_path, paper_img_path]:
    bp.utils.file_handling.export_figure(fig, "img_boxplot_auc", path, ["pdf", "png"], dpi=300)

### Interaction Condition x Log Type

In [ ]:
log_types = ["Naive", "Selfreport w/ App", "App"]

data_analysis = cort_features_analysis.reindex(log_types, level="log_type")

pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("prep", "equal_var"), ("test", "mixed_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "cortisol",
        "within": "log_type",
        "between": "condition",
        "subject": "night_id",
        "groupby": "saliva_feature",
        "multicomp": {"method": "bonf", "levels": None},
    },
)

pipeline.apply(data_analysis)
pipeline.display_results(prep=False, posthoc=False)

### Paired Plots

TODO: replace this by the biopsykit implementation after publishing the latest release

In [ ]:
def columnwise_difference(data: pd.DataFrame):
    cols = data.columns
    df = data.values
    r, c = np.triu_indices(df.shape[1], 1)
    new_cols = [cols[i] + " | " + cols[j] for i, j in zip(r, c)]
    return pd.DataFrame(df[:, c] - df[:, r], columns=new_cols, index=data.index)

#### $AUC_G$

In [ ]:
data_increase = cort_features_analysis.reindex(log_types, level="log_type").xs("auc_g", level="saliva_feature")
data_increase = data_increase.unstack("log_type")["cortisol"].dropna()
data_increase = columnwise_difference(data_increase).round(2)
data_increase = data_increase > 0
data_increase = data_increase.apply(lambda df: df.value_counts(normalize=True) * 100)
data_increase = data_increase.reindex([True, False]).round(0)
data_increase.index.name = "increasing"
dict_saliva_export["AUC_G_Log_Type_Increase"] = data_increase
data_increase

In [ ]:
fig, ax = paired_plot_auc(data=cort_features_analysis, saliva_feature="auc_g", log_types=log_types, figsize=(6, 5))

In [ ]:
log_types = ["Naive", "Selfreport w/ App", "App"]

fig, axs = multi_paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_g", log_types=log_types, figsize=(12, 5)
)

for path in [img_path, paper_img_path]:
    bp.utils.file_handling.export_figure(fig, "img_pairedplot_aucg", path, ["pdf", "png"], dpi=300)

#### $AUC_I$

In [ ]:
data_increase = cort_features_analysis.reindex(log_types, level="log_type").xs("auc_i", level="saliva_feature")
data_increase = data_increase.unstack("log_type")["cortisol"].dropna()
data_increase = columnwise_difference(data_increase).round(2)
data_increase = data_increase > 0
data_increase = data_increase.apply(lambda df: df.value_counts(normalize=True) * 100)
data_increase = data_increase.reindex([True, False]).round(0)
data_increase.index.name = "increasing"
dict_saliva_export["AUC_I_Log_Type_Increase"] = data_increase
data_increase

In [ ]:
fig, ax = paired_plot_auc(data=cort_features_analysis, saliva_feature="auc_i", log_types=log_types, figsize=(6, 5))

In [ ]:
log_types = ["Naive", "Selfreport w/ App", "App"]

fig, axs = multi_paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_i", log_types=log_types, figsize=(12, 5)
)

for path in [img_path, paper_img_path]:
    bp.utils.file_handling.export_figure(fig, "img_pairedplot_auci", path, ["pdf", "png"], dpi=300)

### Increase vs. S2 Time Delay

(Kudielka et al. 2003):  
"We found that the larger the time deviation for sample 2 (+30 min), the smaller the observed awakening cortisol increase. If subjects delay sample 2, they obviously miss the peak, and the resulting awakening increase turns out to be smaller."

#### Data Preparation

Get time deviation for S2 (+30 min)

In [ ]:
s2_delay = cort_samples.xs("S2", level="sample")[["time_diff_to_naive_min"]].dropna()

# drop time deviation outlier, i.e., samples that are be closer to S3 or to S1 than S2 (|delay| >= 7.5 min)
# drop_mask = s2_delay["time_diff_to_naive_min"].abs() >= 7.5
# drop_mask = drop_mask[drop_mask]
# s2_delay = s2_delay.drop(drop_mask.index)
s2_delay.columns = ["s2_delay"]

s2_delay.head()

Get cortisol increase between S0 and S2

In [ ]:
cort_inc = cort_samples[["cortisol"]]
cort_inc = cort_inc.xs("S2", level=-1) - cort_inc.xs("S0", level=-1)
cort_inc = cort_inc.join(s2_delay).dropna()
cort_inc.head()

#### Linear Regression

In [ ]:
data_grp = cort_inc.groupby("log_type")

data_result = {}

for log_type in ["Selfreport w/o App", "App", "Sensor + App"]:
    data_reg = data_grp.get_group(log_type)
    reg = pg.regression.linear_regression(
        X=data_reg["s2_delay"],
        y=data_reg["cortisol"],
    )
    data_result[log_type] = reg

pd.concat(data_result)

#### Regression Plot

In [ ]:
fig, ax = plt.subplots()

for log_type in ["Selfreport w/o App", "App", "Sensor + App"]:
    sns.regplot(
        data=cort_inc.xs(log_type, level="log_type").reset_index(), x="s2_delay", y="cortisol", ax=ax, label=log_type
    )
ax.legend()
fig.tight_layout()

## Export

In [ ]:
bp.io.write_pandas_dict_excel(dict_saliva_export, result_path.joinpath("saliva_results.xlsx"))